In [ ]:
import cryptocompare
import time
import datetime
import pandas as pd
import numpy as np
import os
import shutil
import pyupbit
import requests

In [ ]:
CCList = [
  'BORA', 
  'BTC', 
  'DOGE',
  'ETH', 
  'FLOW',
  'MANA',
  'SAND',
  'SOL',
  'STRK',
  'WEMIX',
  'XRP', 
]
#  'LUNA',
#  'KLAY',
#  'TEMCO', 
# 코빗에는 TEMCO와 BORA가 없음
# 빗썸 거래량 상위 5위와 이름 들어본 것들
# 22. 2. 6. 기준 빗썸 24시간 거래금액 1억 이상
EX = 'Upbit'

In [ ]:
if not os.path.exists('data'):
    os.mkdir('data')

In [ ]:
for CC in CCList:
    filename = "data/{}_{}_KRW_min_gmt.csv".format(EX, CC)
    print("collecting {}...".format(filename))
    if os.path.isfile(filename):
        dt_parser = lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S%z")
        hist_min = pd.read_csv(
            filename, 
            index_col = 0, 
            parse_dates = True, 
            date_parser = dt_parser)
        fromTs = np.max(hist_min.index)
        print("past data file existed, using fromTs:{}". format(fromTs))
        shutil.copy(filename, filename+".bak")
    else:
        hist_min = None
        fromTs = None
        print("no past data file exists")

    # 120분전 datetime을 구한다. 120분전까지만의 분봉을 얻는다.
    # 현재 120분전(2시간) 아무 근거 없음...
    toTs = datetime.datetime.now()
    toTs -= datetime.timedelta(minutes=120)
    apiHist_min = pyupbit.get_ohlcv_from('KRW-{}'.format(CC), 'minute1', fromTs, toTs, period=0.2)
    if apiHist_min is None:
        print("UPbit API failed!!!")
        time.sleep(5)
        # 1번만 더 재시도해본다
        apiHist_min = pyupbit.get_ohlcv_from('KRW-{}'.format(CC), 'minute1', fromTs, toTs, period=0.2)
        if apiHist_min is None:
            print("UPbit API failed again!!!")
            continue
    apiHist_min.index = apiHist_min.index.tz_localize('Asia/Seoul').tz_convert('UTC')

    hist_min_total = pd.concat([hist_min, apiHist_min])

    if hist_min is not None:
        print(f'{len(hist_min.index)=}')
    if apiHist_min is not None:
        print(f'{len(apiHist_min.index)=}')
    print(f'{len(hist_min_total.index)=}')

    hist_min_total.to_csv(filename)

    print("saving hour csv file ...")
    hist_hour = hist_min_total.resample('1H').agg({
        'open': 'first',
        'close': 'last',
        'high': 'max',
        'low': 'min',
        'volume': 'sum',
        'value': 'sum',
    })
    hist_hour.to_csv("data/{}_{}_KRW_hour_gmt.csv".format(EX, CC))

    print("saving day csv file ...")
    hist_day = hist_hour.resample('1D').agg({
        'open': 'first',
        'close': 'last',
        'high': 'max',
        'low': 'min',
        'volume': 'sum',
        'value': 'sum',
    })
    hist_day.to_csv("data/{}_{}_KRW_day_gmt.csv".format(EX, CC))
